In [4]:
%%capture
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *

In [5]:
comp = 'titanic'
path = setup_comp(comp, install = 'fastai "timm >= 0.6.2.dev0"')

In [2]:
%%capture
try: import fastai
except ModuleNotFoundError:
    !pip install -Uq fastai

from fastai.imports import *

In [16]:
!ls {path}

gender_submission.csv  test.csv  train.csv


In [7]:
df = pd.read_csv(path/'train.csv', index_col = 'PassengerId')
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
y_train = df['Survived']
X_train = df.drop(['Survived', 'Name'], axis = 1)

In [12]:
from fastai.tabular.all import *

In [13]:
cont, cat = cont_cat_split(X_train, 1)
print(f'Continuous-type columns: {cont}')
print(f'Categorical-type columns: {cat}')

Continuous-type columns: ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
Categorical-type columns: ['Sex', 'Ticket', 'Cabin', 'Embarked']


In [14]:
for col in cat:
    X_train[col] = pd.Categorical(X_train[col])

In [49]:
# Normalize fare and age
for col in ['Age', 'Fare']:
    X_train[col] = ((X_train[col]) - X_train[col].min()) / (X_train[col].max() - X_train[col].min())

In [50]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
1,3,male,0.271174,1,0,A/5 21171,0.014151,NaN,S
2,1,female,0.472229,1,0,PC 17599,0.139136,C85,C
3,3,female,0.321438,0,0,STON/O2. 3101282,0.015469,NaN,S
4,1,female,0.434531,1,0,113803,0.103644,C123,S
5,3,male,0.434531,0,0,373450,0.015713,NaN,S


In [51]:
%%capture
try: import autosklearn
except ModuleNotFoundError:
    !pip install -Uq auto-sklearn

import autosklearn.classification
cls = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task = 120,
    per_run_time_limit = 30,
    memory_limit = None,
    n_jobs = -1)

In [52]:
cls.fit(X_train, y_train)

/root/mambaforge/lib/python3.9/site-packages/autosklearn/metalearning/metalearning/meta_base.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.metafeatures = self.metafeatures.append(metafeatures)
/root/mambaforge/lib/python3.9/site-packages/autosklearn/metalearning/metalearning/meta_base.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.algorithm_runs[metric].append(runs)


[ERROR] [2022-08-22 19:58:12,341:asyncio.events] 
Traceback (most recent call last):
  File "/root/mambaforge/lib/python3.9/site-packages/distributed/client.py", line 1427, in _handle_report
    msgs = await self.scheduler_comm.comm.read()
  File "/root/mambaforge/lib/python3.9/site-packages/distributed/comm/inproc.py", line 211, in read
    raise CommClosedError()
distributed.comm.core.CommClosedError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/mambaforge/lib/python3.9/site-packages/distributed/utils.py", line 799, in wrapper
    return await func(*args, **kwargs)
  File "/root/mambaforge/lib/python3.9/site-packages/distributed/client.py", line 1435, in _handle_report
    await self._reconnect()
  File "/root/mambaforge/lib/python3.9/site-packages/distributed/utils.py", line 799, in wrapper
    return await func(*args, **kwargs)
  File "/root/mambaforge/lib/python3.9/site-packages/distributed/client.py", line 1

AutoSklearnClassifier(memory_limit=None, n_jobs=-1, per_run_time_limit=30,
                      time_left_for_this_task=120)

In [53]:
print(cls.leaderboard())

          rank  ensemble_weight                 type      cost  duration
model_id                                                                
7            1             0.04        liblinear_svc  0.189831  1.055646
30           2             0.04    gradient_boosting  0.189831  2.754151
29           3             0.08             adaboost  0.189831  2.603024
23           4             0.08             adaboost  0.193220  2.308492
10           5             0.02          extra_trees  0.193220  3.470677
4            6             0.02                  lda  0.200000  1.475597
17           7             0.02                  mlp  0.200000  3.083011
24           8             0.02             adaboost  0.200000  2.308952
31           9             0.02        random_forest  0.203390  2.158354
19          10             0.22             adaboost  0.203390  2.720817
28          11             0.12             adaboost  0.203390  1.023780
35          12             0.02                  qd

In [58]:
df_test = pd.read_csv(path/'test.csv', index_col = 'PassengerId')
df_test = df_test.drop('Name', axis = 1)
for col in cat:
    df_test[col] = pd.Categorical(df_test[col])

for col in ['Age', 'Fare']:
    df_test[col] = ((df_test[col]) - df_test[col].min()) / (df_test[col].max() - df_test[col].min())
df_test.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
892,3,male,0.452723,0,0,330911,0.015282,NaN,Q
893,3,female,0.617566,1,0,363272,0.013663,NaN,S
894,2,male,0.815377,0,0,240276,0.018909,NaN,Q
895,3,male,0.353818,0,0,315154,0.016908,NaN,S
896,3,female,0.287881,1,1,3101298,0.023984,NaN,S


In [59]:
prediction = cls.predict(df_test)

In [60]:
subm = pd.DataFrame(prediction, index = df_test.index, columns = ['Survived'])

In [61]:
subm.to_csv('subm.csv')
!head subm.csv

PassengerId,Survived
892,0
893,0
894,0
895,0
896,0
897,0
898,1
899,0
900,1


In [63]:
# Submit to competition
from kaggle import api
api.competition_submit_cli('subm.csv', 'autoML 120s - normalized', comp)

100%|██████████| 2.77k/2.77k [00:00<00:00, 6.82kB/s]


Successfully submitted to Titanic - Machine Learning from Disaster